In [49]:
from joblib import Parallel, delayed  
import multiprocessing

num_cores = multiprocessing.cpu_count()
num_cores

4

# Joblib Examples

In [50]:
def squareNumber(i):  
    return i * i

In [51]:
inputs = range(25)
Parallel(n_jobs=num_cores)(delayed(squareNumber)(i) for i in inputs)  

[0,
 1,
 4,
 9,
 16,
 25,
 36,
 49,
 64,
 81,
 100,
 121,
 144,
 169,
 196,
 225,
 256,
 289,
 324,
 361,
 400,
 441,
 484,
 529,
 576]

## Check if line segments overlap

In [52]:
import numpy as np
from matplotlib.path import Path

## Create pairs of points for line segments
all_segments = zip(np.random.rand(5000,2),np.random.rand(5000,2))
test_segments = zip(np.random.rand(200,2),np.random.rand(200,2))

## Check if one line segment contains another. 
def check_paths(path):
    for other_path in all_segments:
        res='no cross'
        chck = Path(other_path)
        if chck.contains_path(path)==1:
            res= 'cross'
            break
    return res


In [53]:
%time res = Parallel(n_jobs=-1) (delayed(check_paths) (Path(test_segment)) for test_segment in test_segments)

CPU times: user 112 ms, sys: 26.1 ms, total: 138 ms
Wall time: 4.59 s


In [54]:
%time res = Parallel(n_jobs=1) (delayed(check_paths) (Path(test_segment)) for test_segment in test_segments)

CPU times: user 14.3 s, sys: 289 ms, total: 14.6 s
Wall time: 15 s


## Global States

In [55]:
global A
A = [0, 0, 0, 0]

def modifyArray(i):  
    global A
    A[i] = i
    
A = [0, 0, 0, 0]
for i in range(0, 4):
    modifyArray(i)
    
A

[0, 1, 2, 3]

In [56]:
A = [0, 0, 0, 0]

Parallel(n_jobs=-1) (delayed(modifyArray)(i) for i in range(0, 4))

A

[0, 0, 0, 0]

In [57]:
def calcNewValue(i):
    return i

A = [0, 0, 0, 0]

newA = Parallel(n_jobs=-1) (delayed(calcNewValue)(i) for i in range(0, 4))
newA

[0, 1, 2, 3]

# Machine Learning Examples

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn import datasets

In [59]:
digits = datasets.load_digits()
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(digits.data, digits.target, test_size = 0.35)

## Random Forest

In [60]:
clf = RandomForestClassifier(n_estimators = 200, n_jobs = -1)
clf_base = RandomForestClassifier(n_estimators = 200, n_jobs = 1)

In [61]:
%time res = clf.fit(X_train, Y_train)

CPU times: user 344 ms, sys: 29.8 ms, total: 374 ms
Wall time: 174 ms


In [62]:
%time res = clf_base.fit(X_train, Y_train)

CPU times: user 335 ms, sys: 11.4 ms, total: 346 ms
Wall time: 365 ms


## GridSearch

In [63]:
from sklearn.svm import SVC
from sklearn import grid_search

In [64]:
parameters = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10]}
svr = SVC()

In [65]:
%time res = grid_search.GridSearchCV(svr, parameters, n_jobs = -1).fit(X_train, Y_train)

CPU times: user 113 ms, sys: 28.6 ms, total: 142 ms
Wall time: 632 ms


In [66]:
%time res = grid_search.GridSearchCV(svr, parameters, n_jobs = 1).fit(X_train, Y_train)

CPU times: user 1.3 s, sys: 25.7 ms, total: 1.33 s
Wall time: 1.37 s


# GPU Example

In [67]:
X_train_flat = (X_train/X_train.max())-0.5
X_train_flat = X_train_flat.astype("float32")
Y_train_flat = Y_train.astype('int32')

In [68]:
from lasagne import layers
from lasagne.updates import nesterov_momentum
from lasagne.nonlinearities import softmax
from nolearn.lasagne import NeuralNet

net1 = NeuralNet(
	layers = [
		('input', layers.InputLayer),
		('hidden', layers.DenseLayer),
		('output', layers.DenseLayer),
		],
		#layers parameters:
		input_shape = (None, 64),
		hidden_num_units = 100,
		output_nonlinearity = softmax,
		output_num_units = 10,
 
		#optimization parameters:
		update = nesterov_momentum,
		update_learning_rate = 0.01,
		update_momentum = 0.9,
 
		regression = False,
		max_epochs = 20,
		verbose = 1,
		)

In [69]:
net1.fit(X_train_flat, Y_train_flat)

  input             	(None, 64)          	produces      64 outputs
  hidden            	(None, 100)         	produces     100 outputs
  output            	(None, 10)          	produces      10 outputs
  epoch    train loss    valid loss    train/val    valid acc  dur
-------  ------------  ------------  -----------  -----------  -----
      1       2.33249       2.23909      1.04172      0.16776  0.00s
      2       2.16889       2.06223      1.05172      0.39858  0.00s
      3       1.99281       1.89214      1.05320      0.66321  0.00s
      4       1.81908       1.72251      1.05606      0.72301  0.00s
      5       1.64260       1.55307      1.05765      0.76072  0.00s
      6       1.46758       1.38959      1.05612      0.77244  0.00s
      7       1.30117       1.23676      1.05208      0.79261  0.00s
      8       1.14755       1.09931      1.04388      0.82770  0.00s
      9       1.01099       0.98018      1.03144      0.84723  0.00s
     10       0.89320       0.87975      1

NeuralNet(X_tensor_type=<function matrix at 0x108dfb578>,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x1094c9f50>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x1094bb110>,
     custom_score=None, eval_size=0.2, hidden_num_units=100,
     input_shape=(None, 64),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=20, more_params={},
     objective=<class 'lasagne.objectives.Objective'>,
     objective_loss_function=<function categorical_crossentropy at 0x1094a5cf8>,
     on_epoch_finished=[<nolearn.lasagne.util.PrintLog instance at 0x10a7f3f38>],
     on_training_finished=[],
     output_nonlinearity=<function softmax at 0x1093128c0>,
     output_num_units=10, regression=False,
     update=<function nesterov_momentum at 0x1094ac6e0>,
     update_learning_rate=0.01, update_momentum